# Amazon Augmented AI (Amazon A2I)

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

You can create your own workflows for ML models built on Amazon SageMaker or any other tools. Using Amazon A2I, you can allow human reviewers to step in when a model is unable to make a high confidence prediction or to audit its predictions on an on-going basis. 

Learn more here: https://aws.amazon.com/augmented-ai/

# Integrate human reviewers in Amazon Comprehend Text Classification

To incorporate Amazon A2I into your human review workflows, you need three resources:

* **Worker Task Template** to create a Human Task UI for the worker. The worker UI displays your input data, such as documents or images, and instructions to workers. It also provides interactive tools that the worker uses to complete your tasks. For more information, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html

* **Flow Definition** to create a Human Review Workflow.You use the flow definition to configure your human workforce and provide information about how to accomplish the human review task. You can create a flow definition in the Amazon Augmented AI console or with Amazon A2I APIs. To learn more about both of these options, see https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html

* **Human Loop** starts your human review workflow. When you use one of the built-in task types, the corresponding AWS service creates and starts a human loop on your behalf when the conditions specified in your flow definition are met or for each object if no conditions were specified. When a human loop is triggered, human review tasks are sent to the workers as specified in the flow definition.

When using a custom task type, as this notebook will show, you start a human loop using the Amazon Augmented AI Runtime API. When you call StartHumanLoop in your custom application, a task is sent to human reviewers.

### Install Latest SDK

In [1]:
# First, let's get the latest installations of our dependencies
!pip install -qU pip
!pip install -qU boto3
!pip install -qU botocore

## Setup
We need to set up the following data:
* `region` - Region to call A2I
* `bucket` - A S3 bucket accessible by the given role
    * Used to store the sample images & output results
    * Must be within the same region A2I is called from
* `role` - The IAM role used as part of StartHumanLoop. By default, this notebook will use the execution role
* `workteam` - Subsets or Group of people within your workorce to send the work to

### Role and Permissions

The AWS IAM Role used to execute the notebook needs to have the following permissions:

* ComprehendFullAccess
* SagemakerFullAccess
* S3 Read/Write Access to the BUCKET listed above
* AmazonSageMakerMechanicalTurkAccess (if using MechanicalTurk as your Workforce)

#### Setup Bucket and Paths

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3://sagemaker-us-east-1-835319576252/a2i-results


In [4]:
output_path = f's3://{bucket}/a2i-results'
print(output_path)

s3://sagemaker-us-east-1-835319576252/a2i-results


### Workteam or Workforce

A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers. 

When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job.

To create your Workteam, navigate here:  https://console.aws.amazon.com/sagemaker/groundtruth#/labeling-workforces/create


After you have created your workteam, replace YOUR_WORKTEAM_ARN below

In [5]:
WORKTEAM_ARN = 'arn:aws:sagemaker:us-east-1:835319576252:workteam/private-crowd/dsoaws'

Visit: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html to add the necessary permissions to your role

## Client Setup

Here we are going to setup the rest of our clients. 

In [6]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon SageMaker client
sagemaker = boto3.client('sagemaker', region)

# Amazon Comprehend client
comprehend = boto3.client('comprehend', region)

# Amazon Augment AI (A2I) client
a2i = boto3.client('sagemaker-a2i-runtime')

# Amazon S3 client
s3 = boto3.client('s3', region)

## Create Control Plane Resources

### Create the Human Task UI using a Worker Task Template

Create a human task UI resource, giving a UI template.  This template will be rendered to the human workers whenever human interaction is required.

Below we've provided a simple demo template that is compatible with AWS Comprehend's Detect Sentiment API.

For other pre-built UIs (70+), check: https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [10]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
    <crowd-classifier
      name="sentiment"
      categories="['Positive', 'Negative', 'Neutral', 'Mixed']"
      initial-value="{{ task.input.initialValue }}"
      header="What sentiment does this text convey?"
    >
      <classification-target>
        {{ task.input.taskObject }}
      </classification-target>
      
      <full-instructions header="Text Classification Instructions">
        <p><strong>Positive</strong> sentiment include: joy, excitement, delight</p>
        <p><strong>Negative</strong> sentiment include: anger, sarcasm, anxiety</p>
        <p><strong>Neutral</strong>: neither positive or negative, such as stating a fact</p>
        <p><strong>Mixed</strong>: when the sentiment is mixed</p>
      </full-instructions>

      <short-instructions>
       Choose the primary sentiment that is expressed by the text.
      </short-instructions>
    </crowd-classifier>
</crowd-form>
"""

def create_task_ui():
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker.create_human_task_ui(
        HumanTaskUiName=taskUIName,
        UiTemplate={'Content': template})
    return response

In [9]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'ui-comprehend-' + str(uuid.uuid4()) 

# Create task UI
humanTaskUiResponse = create_task_ui()
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

arn:aws:sagemaker:us-east-1:835319576252:human-task-ui/ui-comprehend-a48f9914-0a95-4dd4-b4c7-bcb1d75bbb8a


### Creating the Flow Definition

In this section, we're going to create a flow definition definition. Flow Definitions allow us to specify:

* The workforce that your tasks will be sent to.
* The instructions that your workforce will receive. This is called a worker task template.
* The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks.
* Where your output data will be stored.

This demo is going to use the API, but you can optionally create this workflow definition in the console as well. 

For more details and instructions, see: https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html.

In [11]:
# Flow definition name - this value is unique per account and region. You can also provide your own value here.
flowDefinitionName = 'fd-dsoaws-comprehend-' + str(uuid.uuid4()) 

create_workflow_definition_response = sagemaker.create_flow_definition(
        FlowDefinitionName= flowDefinitionName,
        RoleArn= role,
        HumanLoopConfig= {
            "WorkteamArn": WORKTEAM_ARN,
            "HumanTaskUiArn": humanTaskUiArn,
            "TaskCount": 1,
            "TaskDescription": "Identify the sentiment of the provided text",
            "TaskTitle": "Classify Sentiment of Text"
        },
        OutputConfig={
            "S3OutputPath" : output_path
        }
    )
flowDefinitionArn = create_workflow_definition_response['FlowDefinitionArn']

In [12]:
# Describe flow definition - status should be active
for x in range(60):
    describeFlowDefinitionResponse = sagemaker.describe_flow_definition(FlowDefinitionName=flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

Initializing
Active
Flow Definition is active


## Human Loops

### Detect Sentiment with AWS Comprehend

Now that we have setup our Flow Definition, we are ready to call AWS Comprehend and start our human loops. In this tutorial, we are interested in starting a HumanLoop only if the SentimentScore returned by AWS Comprehend is less than 99%. 

So, with a bit of logic, we can check the response for each call to Detect Sentiment, and if the SentimentScore is less than 99%, we will kick off a HumanLoop to engage our workforce for a human review. 

# Sample Data

In [19]:
sample_reviews = ['I enjoy this product', 
                  'I am unhappy with this product', 
                  'It is okay', 
                  'sometimes it works']

# Comprehend helper method

In [7]:
# Will help us parse Detect Sentiment API responses 
def capsToCamel(all_caps_string):
    if all_caps_string == 'POSITIVE':
        return 'Positive'
    elif all_caps_string == 'NEGATIVE': 
        return 'Negative'
    elif all_caps_string == 'NEUTRAL':
        return 'Neutral'

In [20]:
human_loops_started = []

SENTIMENT_SCORE_THRESHOLD = 0.99

for sample_review in sample_reviews:
    # Call AWS Comprehend's Detect Sentiment API
    response = comprehend.detect_sentiment(Text=sample_review, LanguageCode='en')
    sentiment = response['Sentiment']
    
    print(f'Processing sample_review: \"{sample_review}\"')
    
    # Our condition for when we want to engage a human for review
    if (response['SentimentScore'][capsToCamel(sentiment)]< SENTIMENT_SCORE_THRESHOLD):
    
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            "initialValue": sentiment.title(),
            "taskObject": sample_review
        }
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=flowDefinitionArn,
            HumanLoopInput={
                "InputContent": json.dumps(inputContent)
            }
        )
        human_loops_started.append(humanLoopName)
        print(f'SentimentScore of {response["SentimentScore"][capsToCamel(sentiment)]} is less than the threshold of {SENTIMENT_SCORE_THRESHOLD}')
        print(f'Starting human loop with name: {humanLoopName}  \n')
    else:
        print(f'SentimentScore of {response["SentimentScore"][capsToCamel(sentiment)]} is above threshold of {SENTIMENT_SCORE_THRESHOLD}')
        print('No human loop created. \n')

Processing sample_review: "I enjoy this product"
SentimentScore of 0.9996746778488159 is above threshold of 0.99
No human loop created. 

Processing sample_review: "I am unhappy with this product"
SentimentScore of 0.9993742108345032 is above threshold of 0.99
No human loop created. 

Processing sample_review: "It is okay"
SentimentScore of 0.5957630276679993 is less than the threshold of 0.99
Starting human loop with name: 2154327b-adf5-4012-a040-2ae4e926d19e  

Processing sample_review: "sometimes it works"
SentimentScore of 0.5166444182395935 is less than the threshold of 0.99
Starting human loop with name: b5d87850-dd6d-4c2d-9174-62c69c509e71  



### Check Status of Human Loop

In [15]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('\n')
    
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 1de31c7a-9f2d-4ad5-9275-87830774bc96
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-835319576252/a2i-results/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c/2020/05/18/19/15/06/1de31c7a-9f2d-4ad5-9275-87830774bc96/output.json'}


HumanLoop Name: 2451d53b-581d-4e82-96ff-847dddabb63e
HumanLoop Status: InProgress
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-835319576252/a2i-results/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c/2020/05/18/19/15/06/2451d53b-581d-4e82-96ff-847dddabb63e/output.json'}




# Wait For Human Workers to Complete Their Tasks
Navigate to the link below and login with your email and password that you used when you setup the Private Workforce.

In [16]:
workteamName = WORKTEAM_ARN[WORKTEAM_ARN.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!
https://bzxy1zzrn2.labeling.us-east-1.sagemaker.aws


### Check Status of Human Loop Again

In [17]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('\n')
    
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 1de31c7a-9f2d-4ad5-9275-87830774bc96
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-835319576252/a2i-results/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c/2020/05/18/19/15/06/1de31c7a-9f2d-4ad5-9275-87830774bc96/output.json'}


HumanLoop Name: 2451d53b-581d-4e82-96ff-847dddabb63e
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-835319576252/a2i-results/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c/2020/05/18/19/15/06/2451d53b-581d-4e82-96ff-847dddabb63e/output.json'}




### View Task Results  

Once work is completed, Amazon A2I stores results in your S3 bucket and sends a Cloudwatch event. Your results should be available in the S3 OUTPUT_PATH when all work is completed.

In [18]:
import re
import pprint

pp = pprint.PrettyPrinter(indent=4)

for resp in completed_human_loops:
    splitted_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = splitted_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response["Body"].read()
    json_output = json.loads(content)
    pp.pprint(json_output)
    print('\n')

{   'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:835319576252:flow-definition/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c',
    'humanAnswers': [   {   'answerContent': {   'sentiment': {   'label': 'Neutral'}},
                            'submissionTime': '2020-05-18T19:19:07.688Z',
                            'workerId': '31199406719ea510'}],
    'humanLoopName': '1de31c7a-9f2d-4ad5-9275-87830774bc96',
    'inputContent': {'initialValue': 'Neutral', 'taskObject': 'It is okay'}}


{   'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:835319576252:flow-definition/fd-dsoaws-comprehend-7467ceeb-882c-4a4d-8221-867080061a3c',
    'humanAnswers': [   {   'answerContent': {'sentiment': {'label': 'Mixed'}},
                            'submissionTime': '2020-05-18T19:19:03.253Z',
                            'workerId': '31199406719ea510'}],
    'humanLoopName': '2451d53b-581d-4e82-96ff-847dddabb63e',
    'inputContent': {   'initialValue': 'Positive',
                     

### The End!